## XGBSurv AH Benchmark

In [1]:
from xgbsurv.datasets import (load_metabric, load_flchain, load_rgbsg, load_support, load_tcga)
from xgbsurv import XGBSurv
from xgbsurv.evaluation import cindex_censored, ibs
from xgbsurv.models.utils import sort_X_y
import os
import numpy as np
import pandas as pd
from scipy.stats import uniform as scuniform
from scipy.stats import randint as scrandint
from scipy.stats import loguniform as scloguniform 
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.decomposition import PCA
# import models
from xgbsurv.models.breslow_final import breslow_likelihood,get_cumulative_hazard_function_breslow
from xgbsurv.models.efron_final import efron_likelihood
from xgbsurv.models.cind_final import cind_loss
from xgbsurv.models.deephit_pycox_final import deephit_loss1_pycox
from xgbsurv.models.eh_aft_final import aft_likelihood
from xgbsurv.models.eh_ah_final import ah_likelihood
from pycox.evaluation import EvalSurv
from sklearn.utils.fixes import loguniform
from xgbsurv.models.utils import sort_X_y_pandas, transform_back, transform
from xgbsurv.preprocessing.dataset_preprocessing import discretizer_df
import os
import sys
current_path = os.getcwd() 
one_level_up = os.path.abspath(os.path.join(current_path,  ".."))
two_levels_up = os.path.abspath(os.path.join(current_path,  "..",".."))
sys.path.append(one_level_up+'/dl_pipeline')
from dl_pipeline import train_dl_complete

## Set Parameters

In [2]:
# set parameters
n_outer_splits = 5
n_inner_splits = 5
rand_state = 42
n_iter = 50
early_stopping_rounds=10
base_score = 0.0
validation_size = 0.2
method = '_dl_'
model = 'ah'

# set seed for scipy
np.random.seed(rand_state)


## Custom Splitting

In [3]:
class CustomSplit(StratifiedKFold):
    def __init__(self, n_splits=5, shuffle=True, random_state=None):
        super().__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def split(self, X, y, groups=None):
        try:
            if y.shape[1]>1:
                y = y[:,0]
        except:
            pass
        bins = np.sign(y)
        return super().split(X, bins, groups=groups)

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

outer_custom_cv = CustomSplit(n_splits=n_outer_splits, shuffle=True, random_state=rand_state)


## Training

In [5]:
data_set_fns = [ load_metabric, load_flchain, load_rgbsg, load_support] #
metrics_list = []

for idx, dataset in enumerate(data_set_fns):
    model = 'ah'
    # get name of current dataset
    data = dataset(path=two_levels_up+"/xgbsurv/datasets/data/", as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_dl_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter)
        print(metric)
        metrics_list.append(metric)



METABRIC_adapted.csv
0
model: ah
gradient clipping
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
integration values 232711
integration values 38852
Concordance Index 0.5385500575373993
Integrated Brier Score: 0.19410806130376207
{'model': 'ah', 'dataset': 'METABRIC', 'cindex_train': [0.5304794409312482], 'cindex_test': [0.5385500575373993], 'ibs_train': [0.17754926059350037], 'ibs_test': [0.19410806130376207]}
1
model: ah
gradient clipping
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
integration values 96628
integration values 19001
Concordance Index 0.5730299514248067
Integrated Brier Score: 0.18531371711944603
{'model': 'ah', 'dataset': 'METABRIC', 'cindex_train': [0.5737544260165288], 'cindex_test': [0.5730299514248067], 'ibs_train': [0.17873712104336725], 'ibs_test': [0.18531371711944603]}
2
model: ah
gradient clipping
Fitting 5 folds for each of 50 candida

In [ ]:
data = load_flchain(path=two_levels_up+"/xgbsurv/datasets/data/", as_frame=True)
df = data.data


In [ ]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+'_'+method+'_results.csv',index=False)

In [ ]:
df

,model,dataset,cindex_train,cindex_test,ibs_train,ibs_test
0,ah,FLCHAIN,0.762044,0.754858,0.100389,0.100006
1,ah,FLCHAIN,0.748817,0.753879,0.102721,0.101657
2,ah,FLCHAIN,0.755970,0.719677,0.100362,0.106899
3,ah,FLCHAIN,0.759133,0.765643,0.100816,0.101474
4,ah,FLCHAIN,0.753270,0.741855,0.102933,0.099462
5,ah,RGBSG,0.443032,0.459058,0.201022,0.202358
6,ah,RGBSG,0.510980,0.480345,0.202184,0.208372
7,ah,RGBSG,0.459217,0.419374,0.201347,0.202430
8,ah,RGBSG,0.558357,0.518009,0.204049,0.200126
9,ah,RGBSG,0.455158,0.420795,0.201701,0.201901


## TCGA Train, Test, Evaluation

In [ ]:
cancer_types = [
    'BLCA',
    'BRCA',
    'HNSC',
    'KIRC',
    'LGG',
    'LIHC',
    'LUAD',
    'LUSC',
    'OV',
    'STAD'
    ]

metrics_list = []

for i, cancer_type in enumerate(cancer_types):
    model = 'ah'
    # get name of current dataset
    data = load_tcga(path=two_levels_up+"/xgbsurv/datasets/data/",cancer_type=cancer_type, as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_dl_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter, tcga=True)
        print(metric)
        metrics_list.append(metric)

BLCA_adapted.csv
0
model: ah
gradient clipping
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
integration values 24713
integration values 1959
Concordance Index 0.5751381215469613
Integrated Brier Score: 0.29901402507479746
{'model': 'ah', 'dataset': 'BLCA', 'cindex_train': [0.545337217449021], 'cindex_test': [0.5751381215469613], 'ibs_train': [0.24582309443949604], 'ibs_test': [0.29901402507479746]}
1
model: ah
gradient clipping
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
integration values 24606
integration values 1889
Concordance Index 0.4576734025122884
Integrated Brier Score: 0.3092587728866417
{'model': 'ah', 'dataset': 'BLCA', 'cindex_train': [0.560304210721573], 'cindex_test': [0.4576734025122884], 'ibs_train': [0.24865993246629867], 'ibs_test': [0.3092587728866417]}
2
model: ah
gradient clipping
Fitting 5 folds for each of 50 candidates, totalling 250 

In [ ]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+'_'+method+'_tcga_results.csv',index=False)

In [ ]:
df